ModuleNotFoundError: No module named 'statsmodels'

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 32.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install sweetviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 13.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 28.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 7.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 16.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import sweetviz as sv 
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot

In [ ]:
strt=pd.read_excel("50_Startups.xlsx")
strt.head()

In [ ]:
strt.info()

In [ ]:
strt.plot(kind='box',subplots=True, layout=(3,3),figsize=(20,20))

In [ ]:
strt1=pd.get_dummies(data=strt,columns=['State'])

In [ ]:
strt1.head()

In [ ]:
from scipy import stats
zsc=stats.zscore(strt1)

In [ ]:
zscores=np.abs(zsc)

In [ ]:
filter_zscores=(zscores<3).all(axis=1)

In [ ]:
filtered=strt1[filter_zscores]

In [ ]:
filtered.shape #seems like there are no outliers as we can see after transforming df and trying to eliminate the zvalues above 3 and below -3 no row got eliminated 

In [ ]:
#Lets Try Isolation Forest method 
#clf=IsolationForest(random_state=20,contamination=0.05) #based on domain knowledge the contamination value should be set
#clf.fit(strt1)

In [ ]:
#strt1['scores']=clf.decision_function(strt1)

In [ ]:
#strt1['anamoly']=clf.predict(strt1)

In [ ]:
#strt1

In [ ]:
strt1.corr() #we can observe that there is no multicollinearity problem between the independent variables

In [ ]:
sns.heatmap(strt1.corr(),annot=True)
#we can consider multicollinearity prob when correlation 'r' value is above 0.75 its nominal to consider for multicollinearity
#there is barely any correlation between State_New_York vs Profit which is 0.031 we can drop this column

In [ ]:
strt2=strt1.drop(['State_New York'],axis=1)
strt2.head()

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
columns=strt2.columns
d=scaler.fit_transform(strt2)
normdf=pd.DataFrame(d,columns=columns)
normdf.head()

In [ ]:
sweet_report=sv.analyze(strt2)
sweet_report.show_html('EDA_of_50Startups.html')

In [ ]:
normdf.corr()

In [ ]:
normdf.rename(columns={'R&D Spend':'RD_Spend'}, inplace=True)
normdf.rename(columns={'Marketing Spend':'Marketing_Spend'},inplace=True)
normdf.head()

In [ ]:
model= smf.ols('Profit~RD_Spend+Administration+Marketing_Spend+State_California+State_Florida',data=normdf).fit()
#here had to change the column name as the code was showing error because the smf was treating R as separate in R&D_Spend column


In [ ]:
model.params

In [ ]:
model.tvalues, '\n', model.pvalues

In [ ]:
#lets build model2 only on RD_SPend,ADministration,Marketing_Spend
model2=smf.ols('Profit~RD_Spend+Administration+Marketing_Spend',data=normdf).fit()

In [ ]:
model2.params

In [ ]:
model2.tvalues, '\n',model2.pvalues 

In [ ]:
normdf1=normdf.drop(['State_California','State_Florida'],axis=1)

In [ ]:
normdf1.corr()

In [ ]:
#Let us build model3 by excluding Administration
model3=smf.ols('Profit~Marketing_Spend+RD_Spend',data=normdf1).fit()

In [ ]:
model3.params

In [ ]:
model3.tvalues,'\n', model3.pvalues

In [ ]:
model3.rsquared,model2.rsquared,model.rsquared
#we can see that after removing insignificant columns (Administration,State) the model's accuracy(R2 value) has barely changed

In [ ]:
#Residual plots for Homoscedasticity
def get_standardized_values (vals):
    return (vals-vals.mean())/vals.std()

In [ ]:
plt.scatter(get_standardized_values(model3.fittedvalues),
           get_standardized_values(model3.resid))
plt.title('Residuals plot for Model3')
plt.xlabel('Fittedvalues')
plt.ylabel('Residuals')
plt.show()


In [ ]:
#Quantile-Quantile plot 
qqplot=sm.qqplot(model3.resid,line='q')
plt.title('Normality of Residuals Plot')
plt.show()


In [ ]:
fig=plt.figure(figsize=(15,8))
fig=sm.graphics.plot_regress_exog(model3,'Marketing_Spend',fig=fig)
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,8))
fig=sm.graphics.plot_regress_exog(model3,'RD_Spend',fig=fig)
plt.show()

In [ ]:
model_influence=model3.get_influence()
(c,_)=model_influence.cooks_distance
summary_cooks=model_influence.summary_frame()
summary_cooks

In [ ]:
fig=plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(normdf1)),np.round(c,3))
plt.title("influential points")
plt.xlabel("row index")
plt.ylabel("Cook's Distance")
plt.show()

In [ ]:
np.argmax(c),np.max(c)

In [ ]:
pip install tabulate

In [ ]:
from tabulate import tabulate

In [ ]:
table=[['Model No.','Rsquared','Adj_Rsquared','AIC','BIC'],
       ['Model 1 ❌',model.rsquared,model.rsquared_adj,model.aic,model.bic],
       ['Model 2 ❌',model2.rsquared,model2.rsquared_adj,model2.aic,model2.bic],
       ['Model 3 ✔️',model3.rsquared,model3.rsquared_adj,model3.aic,model3.bic]]

In [ ]:
print(tabulate(table,headers='firstrow',tablefmt='grid',showindex=range(1,4)))

In [ ]:
model3.summary()

In [ ]:
pip install lazypredict

In [ ]:
from lazypredict.Supervised import LazyRegressor

In [ ]:
strt.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
strt["State"]=le.fit_transform(strt["State"])

In [ ]:
strt.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(1,2))
d=scaler.fit_transform(strt)
df=pd.DataFrame(d,columns=strt.columns)
df.head()

In [ ]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1:]
Y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.33,random_state=9)


In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
reg=LazyRegressor(verbose=0,ignore_warnings=False,custom_metric=None)
models,predictions= reg.fit(x_train,x_test,y_train,y_test)
print(models)

In [ ]:
pip install xelatex